In [1]:
# Using Revystar XE data
# https://projectblue.blob.core.windows.net/media/Default/Imported%20Publication%20Docs/AHDB%20Cereals%20&%20Oilseeds/Disease/Fungicide%20performance/Fungicide%20performance%20for%20wheat,%20barley%20and%20oilseed%20rape%20(Dec%202024).pdf
# https://ahdb.org.uk/news/latest-fungicide-performance-data-includes-new-options-for-cereals

In [2]:
import numpy as np
from scipy.integrate import odeint
import math
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize

# For simulation
from Functions_base import ic_twofield, t_growing, T69, T75, T32, T39

# For ODEs
from Functions_base import beta_base, Temerge, gamma, g_1D, sigma, mu, v

In [3]:
# Disease system
def dPop(ic,t,eps,severity):
    
    beta = beta_base*severity
    
    S_F,E_F,I_F,R_F,D_F,P_F = ic[:6]
    S_N,E_N,I_N,R_N,D_N,P_N = ic[6:]
    
    A_F = S_F + E_F + I_F + R_F + D_F
    A_N = S_N + E_N + I_N + R_N + D_N
    
    # Biocontrol fields
    if t > Temerge:
        FOL = (1 - eps(t))*gamma*E_F
            
        # Set fungicide dependant transmission rate
        FOI = beta*(I_F + P_F)*(1 - eps(t))

        dS_F = g_1D(A_F,t) - sigma(t)*S_F - (S_F/A_F)*FOI
        dE_F = (S_F/A_F)*FOI - sigma(t)*E_F - FOL
        dI_F = FOL - mu*I_F
        dR_F = sigma(t)*(S_F+E_F)
        dD_F = mu*I_F
        dP_F = -v*P_F
        
    else:
        dS_F = 0
        dE_F = 0
        dI_F = 0
        dR_F = 0
        dD_F = 0
        dP_F = 0

    # Uncontrolled fields
    if t > Temerge:
        
        # Set uncontrolled FOI
        FOI = beta*(I_N + P_N)
        
        dS_N = g_1D(A_N,t) - sigma(t)*S_N - (S_N/A_N)*FOI
        dE_N = (S_N/A_N)*FOI - sigma(t)*E_N - gamma*E_N
        dI_N = gamma*E_N - mu*I_N
        dR_N = sigma(t)*(S_N+E_N)
        dD_N = mu*I_N
        dP_N = -v*P_N
        
    else:
        dS_N = 0
        dE_N = 0
        dI_N = 0
        dR_N = 0
        dD_N = 0
        dP_N = 0

    return [dS_F,dE_F,dI_F,dR_F,dD_F,dP_F, dS_N,dE_N,dI_N,dR_N,dD_N,dP_N]

In [4]:
# Data: Protectant, Revystar XE Septoria. T1 is GS32 and T2 is GS39
data = np.array([24.8561151079137, 13.7410071942446, 11.5827338129496, 7.9136690647482])

# When is data assessed? It doesn't say so we assume usual times

In [5]:
# Set time parameters
idx_6975 = slice(T69-Temerge, T75-Temerge,1)
t_6975 = t_growing[idx_6975]

In [6]:
def fit_for_conditions(paras):
    theta,delta,severity=paras
    
    if any(paras <= 0):
        return np.inf
    
    # Most of the Septoria tritici trials were done at T2 so we fit to that time
    def eps0(t):
        # Fungicide concentration
        if t < T39:
            C = 0
        else:
            C = math.exp(-delta*(t-T39))

        return 0*(1-math.exp(-theta*C))
    
    def eps25(t):
        # Fungicide concentration
        if t < T39:
            C = 0
        else:
            C = math.exp(-delta*(t-T39))

        return 0.25*(1-math.exp(-theta*C))
    
    def eps50(t):
        # Fungicide concentration
        if t < T39:
            C = 0
        else:
            C = math.exp(-delta*(t-T39))

        return 0.5*(1-math.exp(-theta*C))
    
    def eps100(t):
        # Fungicide concentration
        if t < T39:
            C = 0
        else:
            C = math.exp(-delta*(t-T39))

        return 1*(1-math.exp(-theta*C))
    
    # Run simulations for 0,25,50,100
    def get_infection_percent(this_eps):
        pop = odeint(func = dPop, y0 = ic_twofield, t = t_growing,args = (this_eps,severity))[:,:5]
        
        Iprev = pop[:,2]/np.sum(pop,axis=1)
        Iav = 100*np.mean(Iprev[idx_6975])
        
        return Iav
    
    I0 = get_infection_percent(eps0)
    I25 = get_infection_percent(eps25)
    I50 = get_infection_percent(eps50)
    I100 = get_infection_percent(eps100)    
    simulation = np.array([I0,I25,I50,I100])
    print(simulation)
    
    # Find the difference between the data and this fit
    return np.linalg.norm((data - simulation)/data)

# Find the minimum distance i.e. the best fit
x = minimize(fit_for_conditions,[6700,1.11e-2,1],method="Nelder-Mead")
print(x)
x = x["x"]

[3.17984214 1.66015719 0.84636628 0.16274509]
[3.17984214 1.66015719 0.84636628 0.16274509]
[3.17984214 1.66015719 0.84636628 0.16274509]
[3.61756302 1.86248205 0.93422121 0.1738542 ]
[3.4672503  1.79320529 0.90428608 0.17011595]
[3.6686589  1.88598881 0.94434696 0.1751076 ]
[3.93165853 2.0066823  0.99608707 0.18143567]
[4.20733844 2.1327313  1.04971352 0.18786282]
[4.79741142 2.40138716 1.1627876  0.20101635]
[4.79741142 2.40138716 1.1627876  0.20101635]
[5.57468602 2.75402037 1.3091031  0.21733562]
[6.06317088 2.97551433 1.400007   0.22712865]
[7.59935157 3.6750796  1.68338008 0.2562432 ]
[8.44274534 4.06302433 1.83875114 0.27144196]
[11.43927224  5.4799398   2.40108652  0.32303882]
[12.14600442  5.82558308  2.53788152  0.33494092]
[16.84235294  8.27705963  3.51708161  0.41499941]
[19.46637555  9.79562375  4.14037715  0.46260855]
[26.7669617  14.84263839  6.3879962   0.62429187]
[29.10747925 16.79947379  7.35893611  0.69019857]
[35.99377398 24.02769086 11.76539804  1.00257792]
[34.22

[19.78452193 15.96557317 12.54341373  7.22994469]
[19.04541585 15.54697051 12.40535753  7.44918026]
[18.82914677 15.38977007 12.30362481  7.42985857]
[18.98985541 15.49328788 12.35589985  7.41212808]
[18.94169659 15.41414761 12.25591658  7.30178398]
[19.14397205 15.60270981 12.42372707  7.41944289]
[18.92501359 15.51907568 12.45361211  7.58394161]
[19.1270891  15.56368018 12.3689029   7.3537526 ]
[19.22104632 15.6488087  12.44247709  7.40227273]
[19.28257025 15.66410441 12.41976073  7.33725939]
[19.22331771 15.63443882 12.41546451  7.36411018]
[19.10853118 15.55160607 12.36266883  7.35550733]
[19.13666691 15.57590448 12.38260721  7.36718843]
[19.18075473 15.58048679 12.35521845  7.30551278]
[19.15316929 15.59695436 12.40622709  7.390345  ]
[19.05458127 15.5233426  12.35651797  7.37697758]
[19.18115245 15.60662558 12.40065728  7.36721573]
[19.18689873 15.62264165 12.42410647  7.39609114]
[19.1719484  15.607894   12.41028587  7.38547683]
[19.20084019 15.6317275  12.42879957  7.39478213]


In [7]:
# Recheck, running from the previous solution
x = minimize(fit_for_conditions,x,method="Nelder-Mead")
print(x)
x = x["x"]

[19.15390048 15.59306972 12.39883927  7.37983094]
[19.15390048 15.52962857 12.28785516  7.22884671]
[19.15390048 15.75511726 12.68397974  7.77535117]
[21.11247739 17.348453   13.89536351  8.32936767]
[17.16776257 13.92056052 11.05924969  6.63313028]
[18.16226421 14.77068594 11.75358029  7.04283658]
[20.13857323 16.4878252  13.1738747   7.89177905]
[18.65870434 15.19846893 12.10511949  7.25153513]
[18.8239312  15.11577203 11.83074519  6.79435121]
[19.07147537 15.59908697 12.47579033  7.53186709]
[19.59262341 15.94922047 12.66956795  7.50607134]
[19.35974349 15.7619357  12.52902839  7.44361797]
[19.23627516 15.77481501 12.65172173  7.68281839]
[19.17449746 15.59068638 12.37806854  7.34007356]
[18.9064882  15.42676107 12.30622435  7.39027547]
[19.08521581 15.47235995 12.24284033  7.20536191]
[19.07491033 15.5683755  12.41907049  7.45160064]
[19.36202845 15.74017865 12.48906678  7.38775761]
[19.24828443 15.66233434 12.44423299  7.38957874]
[19.30945574 15.66004537 12.39112034  7.28360685]
